In [1]:
from libraries import * 
from getData import * 

In [77]:
from libraries import *
from getData import *
from models import *
from params import *
from utils import *

class Model:
    def __init__(self, settings):
        self.cv = settings["Cross Validation Type"]
        self.metric = settings["metric"]
        self.repeat_features = settings["repeat features"]
        self.m = settings["model"]
        self.verbose = settings["verbose"]
        if self.cv != "kTkV" and self.metric != "all-metrics":
            print("original model = ", settings["model"], " ---- ")
            self.model = self.get_model(settings["model"])
        self.pca = PCA(n_components=2)
        self.stratified = settings["stratified"]
        # self.order = settings["order"]
        self.same_split = settings["same split"]
        self.num_features = settings["top_k"]
        
        self.feature_reduction = settings["feature reduction"]
        self.FR_step = settings["features reduced per step"]
        
        self.source = settings["data"]
        self.feature_f = open("features.txt", "w")
        self.init(settings)
        # self.feature_f.close()

    def reduce(self):
        self.outputs = self.outputs.reshape(self.outputs.shape[0], 1)
        self.data = np.hstack((self.data, self.outputs))
        self.data = pd.DataFrame(self.data)

        corrs = self.data.corr().abs()
        importances = corrs.values[-1][:-1]

        top_columns = np.argpartition(importances, -self.num_features)[-self.num_features:]

        self.data = self.data.iloc[:,top_columns]
        self.outputs = self.outputs.ravel()
        self.data = self.data.values
        

    def init(self, augmentation_settings): # gets the data to train and test
        
        # class to get the data from 'getData.py'
        GET = getData(augmentation_settings) 
        
        self.all_features, self.data, self.outputs = GET.data()
        self.all_features = pd.Index(self.all_features)
        
        print("data shape = ", self.data.shape)
        print("outputs shape = ", self.outputs.shape)
        self.outputs = self.outputs.ravel()
    
        self.forward()
        # Check for stratified sampling part later 
        
        # if self.stratified == True:
        #     self.data = pd.DataFrame(self.data)
        #     self.data["outputs"] = self.outputs 
        #     # if self.order =="ascending":
        #     self.data = self.data.sort_values(by = "outputs", ascending = True)
        #     # elif self.order == "descending":
        #         # self.data = self.data.sort_values(by = "outputs", ascending = False)
        #     self.outputs = self.data["outputs"].values
        #     self.data = self.data.drop(["outputs"], axis = 1)
        #     self.data = self.data.values
            
        # self.forward()


    def CV_(self):
        if self.cv == "LOO":
            return 55
        elif self.cv == "LFiveO":
            return 11
        elif self.cv == "5Fold":
            return 5
        elif self.cv == "kTkV":
            return 10

    def correct_metric(self):
        if self.metric == "MSE":
            self.metric = "neg_mean_squared_error"
        elif self.metric == "MAE":
            self.metric = "neg_mean_absolute_error"
            

    def get_model(self, model, params=None):

        m = getModel(model)
        params = allParams(model) if params == None else params

        if self.cv != "kTkV" and self.metric != "all-metrics":
            self.correct_metric()
            return GridSearchCV( m(), params, cv= self.CV_(), scoring = self.metric , return_train_score = True, verbose = self.verbose)
        else:
            params = dict(zip(self.param_keys, params))
            return m(**params)
            # return make_pipeline(
            #                     StandardScaler(),
            #                     m(**params))

##################################################################################################################################################################
# the following functions are not necessary as of now
##################################################################################################################################################################
    #
    # def train(self, data, outputs):
    #     self.model.fit(data, outputs)
    #
    # def predict(self, data):
    #     return self.model.predict(data)
    #
    # def get_score(self,data,outputs, return_ = False): #gives out R2 scores
    #     print("R2 fit score = ", self.model.score(data, outputs), "\n" )
    #     if return_:
    #         return self.model.score(data, outputs)
    #
    # def CrossValResults(self):
    #     print(self.model.cv_results_)
    #     print('\n')
    #
    # def modify_data(self, data):
    #     self.pca.fit(data)
    #     raise NotImplementedError()
    #
    # def get_data(self):
    #     X_train, X_test, y_train, y_test = train_test_split( self.data, self.outputs, test_size=0.09, random_state=42)
    #
    #     if self.cv != "kTkV":
    #         X_train, y_train = self.data, self.outputs
    #
    #     print("train data shape = ", X_train.shape)
    #     print("train labels shape = ", y_train.shape, "\n")
    #
    #     print("test data shape = ", X_test.shape)
    #     print("test labels shape = ", y_test.shape, "\n")
    #
    #     return X_train, X_test, y_train, y_test
    #
    # def oneMetric_cv(self):
    #     X_train, X_test, y_train, y_test = self.get_data()
    #     self.train(X_train, y_train)
    #
    #     print("training performance")
    #     self.get_score(X_train, y_train)
    #     self.train_predictions_ = self.predict(X_train)
    #     # self.test_predictions_ = self.predict(X_test)
    #    
##################################################################################################################################################################    



    def important_columns(self, data, outputs): 
        """This function finds the names of important features in the data, either based on two factors
            - Pearson Correlation with WAB scores 
            - Recurrent Feature Elimination (RFE)

        Args:
            data (numpy array): Input data without output column
            outputs (numpy array): 1D array of WAB scores / any other output

        Returns:
            numpy array : finds the column index numbers and returns it 
        """
        # print(self.feature_reduction)
        self.feature_f.write(self.feature_reduction + "\n")
        
        if self.feature_reduction == "pearson":

            outputs = outputs.reshape(outputs.shape[0], 1)
            data = np.hstack((data, outputs))
            data = pd.DataFrame(data)
    
            corrs = data.corr().abs()
            importances = corrs.values[-1][:-1]
    
            # top_columns = np.argpartition(importances, -self.num_features)[-self.num_features:]
            top_columns = heapq.nlargest(150, range(len(importances)), importances.take)
            # print(top_columns[:3])
            return top_columns
            
        elif self.feature_reduction == "RFE":

            self.rfe = RFE(self.model, n_features_to_select = self.num_features, step=1)#self.FR_step)
            self.rfe = self.rfe.fit(data, outputs)
            features = np.arange(data.shape[1])
            features = features[self.rfe.ranking_==1]
            return features

    def reduce_data(self, data):
        """filter important columns from the input data

        Args:
            data (numpy array): Input data without output columns

        Returns:
            numpy array: Input data with selected top features
        """
        
        data = pd.DataFrame(data)
        # print("dataframe")
        data = data.iloc[:,self.top_columns]
        # print("few columns")
        return data.values
        
        
    def get_KFfname(self):
        """create a csv file name based on model parameter setting

        Returns:
            string: csv file name based on the model parameters 
        """
        
        fname = ""
        for i,param in enumerate(self.param_keys):
            fname += "[" + param[0] + "-" + str(self.curr_param[i]) + "]_" 
        fname = fname[:-1] + ".csv"
        return fname
        
            
    def get_folder_name(self, fname):
        """creates a folder with the training setting in the results folder

        Args:
            fname (string): csv file name to store model parameter-wise outputs

        Returns:
            string: folder_name + file_name to save model parameter-wise outputs. 
        """
        path = create_folder(self.source, fname)
        stratified = "" if self.stratified!=True else "stratified"
        fname = self.cv + "_" + self.m + "_" + self.metric + "_top" + str(self.num_features) + "frs_" + self.feature_reduction + "_"
        return path + "/" + fname[:-1]
    
    
    def features_init(self):
        """creates a folder to save outputs of important features

            Initiate the csv writer to write the features in a file during cross validation

        """
        folder = self.get_folder_name("features")
        if not os.path.exists(folder):
            os.makedirs(folder,exist_ok=True)
        file = self.get_KFfname()

        self.writerfile = open( folder + "/" + file , 'w', newline='')
        self.feature_writer = csv.writer(self.writerfile, delimiter=' ',  quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        
    def save_features(self):
        """writes the important features to a csv file using the csv-writer
        
        """
        features = self.all_features[self.top_columns]
        self.feature_writer.writerow(list(features.values))
        
    
    def feature_close(self):
        """Closes the csv-writer to complete writing important features to a file
        """
        self.writerfile.close()
        

    def validate(self, data, outputs, kf2 = None):
        """Does Cross valiation for kTkV training setting, Training for other settings

        Args:
            data (numpy array): input data
            outputs (numpy array): WAB scores / output values
            kf2 (cross validation setting, optional): kFold CV object as input. If not given, creates 10-Fold CV. Defaults to None.

        Returns:
            float, float, float, float, object: train MSE mean, Validate MSE mean, train MAE mean, validate MAE mean, Best validate model
        """

        ##################################################################################################################################################################
        # Do not disturb the following few lines on k-fold setting
        if kf2 != None: 
            if self.stratified:
                class_labels = self.get_class_labels(splits = 5, outputs = outputs)
            else:
                class_labels = outputs
        else:
            if self.stratified != True: 
                if self.cv != "LOO":
                    kf2 = KFold(n_splits=self.CV_(), shuffle = True)
                    class_labels = outputs
            else:
                
                kf2 = StratifiedKFold(n_splits=self.CV_(), shuffle = True)
                class_labels = self.get_class_labels(splits = 5, outputs = outputs)

            if self.cv == "LOO":
                kf2 = KFold(n_splits=self.CV_(), shuffle=True)
                class_labels = outputs
                
        ##################################################################################################################################################################
        train_performance = []
        train_performance_MAE = []

        validate_performance = []
        validate_performance_MAE = []

        models = []

        mean_train_MSE = []
        mean_validate_MSE = []
        mean_test_MSE = []
        
        cumulative_validate_predictions = []
        cumulative_ground_truths = []

        maxLoss = 1000 # some arbitrary max loss value for RMSE
        self.best_features = None #to keep track of best features

        for train_index, validate_index in kf2.split(data, class_labels):

            # Model definition
            self.model = self.get_model(self.m, self.curr_param) # gets the model initialization of "m" type and "curr_param" parameters
            
            X_train, X_validate = data[train_index], data[validate_index]
            y_train, y_validate = outputs[train_index], outputs[validate_index]
            # print("got val data")
            
            if self.num_features!=-1: #(self.num_features!=-1 and self.cv!="kTkV"):
                self.top_columns = self.important_columns(X_train,y_train)
                # print("found important columns")
                X_train = self.reduce_data(X_train)
                # print("reduced train data")
                X_validate = self.reduce_data(X_validate)

            # print("split val data")
            # print("important columns = ", self.top_columns)
            self.save_features()

            self.model.fit(X_train, y_train)            
            train_predictions = self.model.predict(X_train)
            # print("validate shape = ", X_validate.shape)
            validate_predictions = self.model.predict(X_validate)
            # return
            # print("got val preds")
            
            if self.cv != "kTkV":
                cumulative_validate_predictions.append(validate_predictions)
                cumulative_ground_truths.append(y_validate)
                
            models.append(self.model)

            if mean_squared_error(validate_predictions,y_validate,squared=False) < maxLoss:
                maxLoss = mean_squared_error(validate_predictions,y_validate,squared=False)
                self.best_features = self.top_columns 
 
            # trainloss = mean_squared_error(train_predictions, y_train, squared=False)
            valloss = mean_squared_error(validate_predictions,y_validate,squared=False)
            self.feature_f.write(str(self.top_columns[:3]) + " -- "+ str(valloss) +"\n")
               
            train_performance.append(mean_squared_error(train_predictions, y_train, squared=False))
            validate_performance.append(mean_squared_error(validate_predictions,y_validate,squared=False))

            train_performance_MAE.append(mean_absolute_error(train_predictions, y_train))
            validate_performance_MAE.append(mean_absolute_error(validate_predictions,y_validate))

        if self.cv != "kTkV":
            self.save_file(cumulative_validate_predictions, cumulative_ground_truths)

        model = models[np.argmin(validate_performance)]
        self.top_columns = self.best_features
        return np.mean(train_performance), np.mean(validate_performance), np.mean(train_performance_MAE), np.mean(validate_performance_MAE), model


    def save_file(self, cumulative_validate_predictions, cumulative_ground_truths):
        """saves the predictions and ground truth values for each model setting for each training setting.

        Args:
            cumulative_validate_predictions (list[float]): list of predictions for each patient
            cumulative_ground_truths (list[float]): list of ground truth scores for each patient
        """
        
        cumulative_validate_predictions = list(itertools.chain(*cumulative_validate_predictions))
        cumulative_ground_truths = list(itertools.chain(*cumulative_ground_truths))
        
        predictions = {
            "predictions" : cumulative_validate_predictions,
            "ground truth score": cumulative_ground_truths,
        }
        folder_name = self.get_folder_name("outputs")
        
        if not os.path.exists(folder_name ):
            os.makedirs(folder_name,exist_ok=True)
        
        predictions = pd.DataFrame(predictions)
        self.kFfname = self.get_KFfname()

        predictions.to_csv(folder_name + "/" + self.kFfname)   
        


    def get_class_labels(self, splits, outputs):
        """Helper function. Used in stratifold setting. Not important for regular setting. 

        Args:
            splits (int): number of buckets in training data during k-Fold CV
            outputs (array[float]): WAB scores of each patient. 

        Returns:
            array[float]: labels of each training example before splitting the data.
        """
        x = np.ones(len(outputs))
        
        counter = 0
        counter_update = int(len(outputs)/splits)
        adder = 0
        
        while(counter < len(outputs)):
            x[counter:counter + counter_update] += adder 
            adder += 1
            counter += counter_update 
            
        return x 
            

    def train_kTkV(self):
        """Training loop for kTkV setting. Initiates Training set and Testing set here. Sends training data to validate function.

        Returns:
            float, float, float, float, float, float, object: trainMSE, testMSE, valMSE, trainMAE, valMAE, testMAE, best test model.
        """
        
        if self.stratified!=True:
            kf = KFold(n_splits=11, shuffle = False)
            class_labels = self.outputs
        else:
            kf = StratifiedKFold(n_splits=11, shuffle = False)
            class_labels = self.get_class_labels(splits=5, outputs = self.outputs)

        
        train_performances = []
        train_performances_MAE = []

        validate_performances = []
        validate_performances_MAE = []

        test_performances = []
        test_performances_MAE = []
        
        cumulative_test_predictions = []
        cumulative_test_truths = []

        models = []

        for train_index, test_index in kf.split(self.data, class_labels):
            X_train, X_test = self.data[train_index], self.data[test_index]
            y_train, y_test = self.outputs[train_index], self.outputs[test_index]
            # print('data split here')
            train_performance, validate_performance, train_performance_MAE, validate_performance_MAE, model = self.validate(X_train, y_train)
            # print("got performances")
            self.feature_f.write(str(self.top_columns[:3]) + "\n")
            X_train = self.reduce_data(X_train)
            X_test = self.reduce_data(X_test)
            # print("data reduced")
            models.append(model)

            test_predictions = model.predict(X_test)
            
            cumulative_test_predictions.append(test_predictions)
            cumulative_test_truths.append(y_test)

            test_performance = mean_squared_error(test_predictions, y_test, squared=False)
            test_performance_MAE = mean_absolute_error(test_predictions, y_test)

            train_performances.append(train_performance)
            train_performances_MAE.append(train_performance_MAE)

            validate_performances.append(validate_performance)
            validate_performances_MAE.append(validate_performance_MAE)

            test_performances.append(test_performance)
            test_performances_MAE.append(test_performance_MAE)
        # print("loop ended")
        self.save_file(cumulative_test_predictions, cumulative_test_truths)
        model = models[np.argmin(test_performances)]

        return np.mean(train_performances), np.mean(validate_performances), np.mean(test_performances), np.mean(train_performances_MAE), np.mean(validate_performances_MAE), np.mean(test_performances_MAE), model


    def get_params_(self):
        """returns the combination of different model parameters

        Returns:
            list: combinations of different parameters.
        """
        self.parameters = allParams(self.m)
        self.param_keys = list(self.parameters)
        
        param_list = list(itertools.product(*self.parameters.values()))        
        return param_list


    def kTkV(self):
        """initiaties the kTkV training setting. Selects a model parameters, then sends the data to train_kTkV to start the training process.
        """

        validate_performances = []
        validate_performances_MAE = []

        test_performances = []
        test_performances_MAE = []

        train_performances = []
        train_performances_MAE = []

        models = []

        param_list = self.get_params_()

        for self.curr_param in tqdm(param_list, "params"):
            self.features_init()
            # print("features initialized")
            self.model = self.get_model(self.m, self.curr_param)

            train_performance, validate_performance, test_performance, train_performance_MAE, validate_performance_MAE, test_performance_MAE, model = self.train_kTkV()
            # print("got performances")
            train_performances.append(train_performance)
            train_performances_MAE.append(train_performance_MAE)

            validate_performances.append(validate_performance)
            validate_performances_MAE.append(validate_performance_MAE)

            test_performances.append(test_performance)
            test_performances_MAE.append(test_performance_MAE)
            
            self.feature_close()
            models.append(model)
            # break 
        
        self.feature_f.close()

        train_performance_ranking = sp.rankdata(train_performances)
        val_performance_ranking = sp.rankdata(validate_performances)
        test_performance_ranking = sp.rankdata(test_performances)


        self.dataframe = {

            "train performance RMSE" : train_performances,
            "test performance RMSE" : test_performances,
            "validate performance RMSE" : validate_performances,

            "train performance MAE" : train_performances_MAE,
            "validate performance MAE" : validate_performances_MAE,
            "test performance MAE" : test_performances_MAE,

            "train rank" : train_performance_ranking,
            "validate rank" : val_performance_ranking,
            "test rank" : test_performance_ranking,
            
            "max depth" : '' if self.m == "SVR" else max([estimator.tree_.max_depth for estimator in model[-1].estimators_])


        }
        self.dataframe["'|'".join(list(self.parameters))] = param_list
        self.cv_results_ = pd.DataFrame.from_dict(self.dataframe)


    def normal_CV(self):
        """Initiates the training setting for normal Cross Validation. Uses validate_kTkV for training.
        """

        train_performances = []
        test_performances = []

        train_performances_MAE = []
        test_performances_MAE = []

        param_list = self.get_params_()

        def train_loop():
            print("we entered training loop")
            for self.curr_param in tqdm(param_list, desc = "params"):
                # print("we here")
                train_performance_MSE, test_performance_MSE, train_performance_MAE, test_performance_MAE, model  = self.validate(self.data, self.outputs, kf)
                train_performances.append(train_performance_MSE)
                test_performances.append(test_performance_MSE)

                train_performances_MAE.append(train_performance_MAE)
                test_performances_MAE.append(test_performance_MAE)

        
            train_performance_rankings_MSE = sp.rankdata(train_performances)
            test_performance_rankings_MSE = sp.rankdata(test_performances)

            train_performance_rankings_MAE = sp.rankdata(train_performances_MAE)
            test_performance_rankings_MAE = sp.rankdata(test_performances_MAE)


            self.dataframe = {
                "train RMSE" : train_performances,
                "train RMSE rank" : train_performance_rankings_MSE,

                "test RMSE" : test_performances,
                "test RMSE rank" : test_performance_rankings_MSE,

                "train MAE" : train_performances_MAE,
                "train MAE rank" : train_performance_rankings_MAE,

                "test MAE" : test_performances_MAE,
                "test MAE rank" : test_performance_rankings_MAE,
                
                "max depth" : '' if self.m == "SVR" else max([estimator.tree_.max_depth for estimator in model.estimators_])
            }

            self.dataframe["'|'".join(list(self.parameters))] = param_list
            self.cv_results_ = pd.DataFrame.from_dict(self.dataframe)

        if self.same_split:
            if self.stratified != True:
                kf = KFold(n_splits=self.CV_(), shuffle = False)
            else:
                kf = StratifiedKFold(n_splits=self.CV_(), shuffle = False)
                if self.cv == "LOO":
                    kf = KFold(n_splits=self.CV_(), shuffle=True) #change this later

        else:
            kf = None
            
        train_loop()


    def forward(self):
        """Based on training setting, this redirects training to one of the initializers.
        """

        if self.cv != "kTkV":
            if self.metric != "all-metrics":
                self.oneMetric_cv()
            elif self.metric == "all-metrics":
                # print("all metric CV")
                self.normal_CV()
    
        else:   
            self.kTkV()



In [80]:
Augmentation_settings = {
    "Cross Validation Type": "kTkV",
    "model" : "RF",
    "metric" : "all-metrics", # not required anymore

    "Augment" : False, # not implemented yet
    # "transform" : True, # need to implement this now. 

    "verbose" : 0,    # dont change this. Taken care of 
    "top_k" : 150,

    "repeat features" : False,
    "same split" :  False, #implement false condition
    
    "stratified" : False,
    "data" : "RS",
    
    "feature reduction" : "pearson",
    "features reduced per step" : 1
    # "order" : args.order if 'stratified' in args else None, #implement false condition
}

if __name__ == "__main__":
    model = Model(Augmentation_settings)
    

loading resting state data
path =  /Users/saurav/Desktop/Margrit/MRI/RS/time_series
data shape =  (55, 1128)
outputs shape =  (55,)


params:   3%|▎         | 2/75 [01:00<36:43, 30.18s/it]


KeyboardInterrupt: 

In [ ]:
np.argmax(corrs.values[-1][:-1])

In [ ]:
# import heapq

# test_top_cols = heapq.nlargest(150, range(len(importances)), importances.take)

# test_top_cols.sort()
# top_columns.sort()
# # test_top_cols

importances.take

## Training loop test

In [66]:
outputs = outputs.flatten()
data.head()



,0,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,-0.360065,-0.007469,0.054958,0.266475,0.232611,-0.338867,0.631779,-0.359282,-0.410621,-0.126857,...,0.102267,0.071445,0.572910,0.096260,0.013795,0.054751,-0.027064,0.266298,-0.134463,87.2
1,0.185745,0.159007,-0.080865,-0.160646,-0.047338,0.138317,0.461644,0.318556,0.088526,-0.203324,...,0.251529,-0.040121,0.591814,0.084357,0.088996,-0.136049,0.073134,0.295827,-0.101401,25.2
2,-0.000432,-0.106150,0.111955,0.033202,0.061812,0.090680,0.145437,-0.058595,-0.129418,-0.399258,...,-0.148758,0.048900,0.243912,-0.075932,0.037224,0.024014,0.164834,0.163829,0.143013,52.0
3,-0.066249,0.105581,-0.195710,-0.257786,-0.055454,-0.130789,0.770839,0.209555,-0.084566,-0.312965,...,0.302855,0.430871,0.405293,0.045807,0.209190,0.094675,-0.078121,0.682280,0.009166,74.1
4,0.047168,0.040613,-0.066341,-0.095488,-0.215614,0.012098,0.140075,0.207200,-0.067715,-0.029938,...,0.257647,0.006619,0.504215,-0.046846,-0.000666,-0.150161,-0.164306,0.597433,-0.112093,30.8


In [72]:

def important_columns(data, outputs, num_features): 
    """This function finds the names of important features in the data, either based on two factors
        - Pearson Correlation with WAB scores 
        - Recurrent Feature Elimination (RFE)

    Args:
        data (numpy array): Input data without output column
        outputs (numpy array): 1D array of WAB scores / any other output

    Returns:
        numpy array : finds the column index numbers and returns it 
    """    

    outputs = outputs.reshape(outputs.shape[0], 1)
    data = np.hstack((data, outputs))
    data = pd.DataFrame(data)

    corrs = data.corr().abs()
    importances = corrs.values[-1][:-1]

    top_columns = np.argpartition(importances, -num_features)[-num_features:]
    return top_columns

def reduce_data( data, top_columns):
    """filter important columns from the input data

    Args:
        data (numpy array): Input data without output columns

    Returns:
        numpy array: Input data with selected top features
    """
    
    data = pd.DataFrame(data)
    data = data.iloc[:,top_columns]
    return data.values
    


In [132]:
# data = data.values
# data = data[:,:-1]
data2 = data

In [135]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel



kf = KFold(n_splits=11, shuffle = True)

num_features = 25
total , count = 0,0

params = {
    "n_estimators": 500,
    "max_depth": 30,
    "min_samples_split": 2,
    "learning_rate": 0.00001,
    "loss": "ls",
}
top_columns = important_columns(data, outputs, num_features)
X_train = reduce_data(data, top_columns)

for train_index, validate_index in kf.split(data, outputs):
    X_train, X_test, y_train, y_test = data[train_index], data[validate_index], outputs[train_index], outputs[validate_index] 
    # top_columns = important_columns(X_train, y_train, num_features)
    # X_train = reduce_data(X_train, top_columns)
    # X_test = reduce_data(X_test, top_columns)
 
    # kernel = DotProduct() + WhiteKernel()
    
    # m1 = GaussianProcessRegressor(kernel=kernel, random_state=0)
    # m1 = ensemble.GradientBoostingRegressor(**params)   
    # m1 = AdaBoostRegressor(RandomForestRegressor(max_depth=5, random_state=0))
    
    m1 = AdaBoostRegressor(SVR(kernel = "linear", C=1, epsilon=10, tol=0.01, gamma=1e-3))
    model = make_pipeline(StandardScaler(), m1)
    model.fit(X_train, y_train)
    
    test_outputs = model.predict(X_test)
    mse = mean_squared_error(test_outputs, y_test, squared=False)
    
    total+=mse 
    count+=1
    
    print(test_outputs, " -- ", mse)
    # print(y_test)

print("mean rmse = ", float(total/count))



[62.8775271  54.88075783 57.47601334 59.65109178 69.44145882]  --  21.441909466093673
[53.46994776 57.41393091 65.89395901 68.84652212 69.37231472]  --  21.521391759865427
[62.01827608 60.58096351 67.89312279 68.89568014 69.38485639]  --  28.89929220535707
[69.24179185 63.92448064 62.98333279 65.3905103  70.14190337]  --  28.975892760547104
[75.99682532 67.75055461 69.39321837 82.03643542 59.97585907]  --  17.316080966391137
[59.0003309  62.6137666  50.97141126 81.02389895 71.55495703]  --  24.40554859589116
[65.49623452 64.95465626 70.10976615 65.15957619 63.91266977]  --  23.374218340396485
[67.50033243 72.92022438 73.15172227 66.69538247 79.690335  ]  --  19.427007203423297
[56.8182744  67.93612582 75.36565606 68.02549464 58.68802928]  --  16.678809835741774
[71.37754764 70.49590017 72.43324831 76.37256812 67.41045608]  --  29.559888127231616
[70.84167949 66.96301448 64.77157511 72.48024486 59.95053366]  --  13.836128534346118
mean rmse =  22.312378890480442
